In [1]:
%load_ext autoreload
%autoreload 2

# Langevin MD Implicit Simulation

In [2]:
import molsysmt as msm
import uibcdf_tools as uibcdf_tools
import numpy as np
from simtk import unit
import simtk.openmm as mm
import simtk.openmm.app as app
from mdtraj.reporters import HDF5Reporter
from sys import stdout

In [ ]:
forcefield = app.AmberPrmtopFile('ff99sb_jacs2002.prmtop')
#topology = msm.convert('ff99sb_jacs2002.prmtop', to_form='openmm.Topology')
topology = forcefield.topology
positions = msm.get('Trp-cage.pdb', coordinates=True)[0]

In [ ]:
system = forcefield.createSystem(implicitSolvent=app.OBC2, soluteDielectric=1.0, solventDielectric=78.5)
# En el tutorial usan solventDielectric=78.5
# Ver http://docs.openmm.org/7.1.0/userguide/application.html#amber-implicit-solvent
# Puedo tambien poner implicitSolventKappa=1.0/nanometer

# El warning que da es por que el parametro radii para un átomo no es el mismo que el considerado por openmm
# como adecuado para OBC2: en el prmtop es 0.12 y debería ser 0.13

In [ ]:
temperature = 325*unit.kelvin
integration_timestep = 2.0*unit.femtosecond
damping = 1.0/unit.picosecond

integrator = mm.LangevinIntegrator(temperature, damping, integration_timestep)

In [ ]:
platform = mm.Platform.getPlatformByName('CUDA')

In [ ]:
properties = {'CudaPrecision': 'mixed'}
#properties["DeviceIndex"] = "0" # We should also specify which GPU we want to use for the simulation

In [ ]:
simulation = app.Simulation(topology, system, integrator, platform, properties)

In [ ]:
simulation.context.setPositions(positions)
simulation.context.setVelocitiesToTemperature(temperature)

In [ ]:
md_time = 5.0 * unit.nanoseconds
saving_time_interval = 10.0 * unit.picoseconds
log_time_interval = 100.0 * unit.picoseconds

md_steps = np.rint(md_time/integration_timestep).astype(int)
saving_steps_interval = np.rint(saving_time_interval/integration_timestep).astype(int)
log_steps_interval = np.rint(log_time_interval/integration_timestep).astype(int)

In [ ]:
reporter_log_stdout = app.StateDataReporter(stdout, log_steps_interval, step=True, time=True,
                                            potentialEnergy=True, kineticEnergy=True,
                                            totalEnergy=True, temperature=True,
                                            progress=True, remainingTime=True,
                                            speed=True, totalSteps=md_steps, separator='\t')


reporter_logfile = app.StateDataReporter('traj.log', log_steps_interval, step=True, time=True,
                                            potentialEnergy=True, kineticEnergy=True,
                                            totalEnergy=True, temperature=True,
                                            progress=True, remainingTime=True,
                                            speed=True, totalSteps=md_steps, separator='\t')

reporter_h5file = HDF5Reporter('traj.h5', saving_steps_interval, coordinates=True, time=True,
                               potentialEnergy=True, kineticEnergy=True, temperature=True)

simulation.reporters.append(reporter_log_stdout)
simulation.reporters.append(reporter_logfile)
simulation.reporters.append(reporter_h5file)

In [ ]:
print(simulation.context.getState(getEnergy=True).getPotentialEnergy())
simulation.minimizeEnergy()
print(simulation.context.getState(getEnergy=True).getPotentialEnergy())

In [ ]:
simulation.step(md_steps)

In [ ]:
reporter_h5file.close()

In [3]:
msm.view('traj.h5')

/home/diego/Myopt/miniconda3/envs/UIBCDF_lab_dev/lib/python3.7/site-packages/tables/leaf.py:544: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  key = numpy.array(key)


NGLWidget(max_frame=499)